In [26]:
import sqlite3
import pandas as pd
from IPython.display import display

In [27]:
database = 'data/mlb_data.sqlite'
conn = sqlite3.connect(database)
cursor = conn.cursor()

In [28]:
query = "SELECT name FROM sqlite_schema WHERE type='table' ORDER BY name"
rows = cursor.execute(query).fetchall()
print(rows)

[('all_star',), ('appearances',), ('batting',), ('batting_postseason',), ('college',), ('fielding',), ('fielding_outfield',), ('fielding_postseason',), ('hall_of_fame',), ('home_game',), ('manager',), ('manager_award',), ('manager_award_vote',), ('manager_half',), ('park',), ('pitching',), ('pitching_postseason',), ('player',), ('player_award',), ('player_award_vote',), ('player_college',), ('postseason',), ('salary',), ('team',), ('team_franchise',), ('team_half',)]


In [31]:
# query = 'SELECT * from appearances'
query = 'SELECT player_id, g_c from appearances'
rows = cursor.execute(query).fetchall()
for row in rows:
    print(row)

('barnero01', 0)
('barrofr01', 0)
('birdsda01', 7)
('conefr01', 0)
('gouldch01', 0)
('jackssa01', 0)
('mcveyca01', 29)
('schafha01', 0)
('spaldal01', 0)
('wrighge01', 0)
('wrighha01', 0)
('brannmi01', 0)
('duffyed01', 0)
('foleyto01', 4)
('hodesch01', 20)
('kingma01', 9)
('mcatebu01', 0)
('pinkhed01', 0)
('simmojo01', 0)
('treacfr01', 0)
('woodji01', 0)
('zettlge01', 0)
('allisar01', 0)
('bassjo01', 1)
('battijo01', 0)
('carleji01', 0)
('ewellge01', 0)
('johnsca01', 0)
('kimbage01', 0)
('paborch01', 0)
('prattal01', 0)
('questjo01', 0)
('suttoez01', 1)
('whitede01', 29)
('whiteel01', 3)
('armstbo01', 0)
('barrebi01', 1)
('biermch01', 0)
('careyto01', 0)
('deaneha01', 0)
('donnepe01', 0)
('foranji01', 0)
('goldswa01', 2)
('halliji01', 0)
('kellybi01', 0)
('kohlehe01', 2)
('lennobi01', 12)
('mathebo01', 0)
('mcderjo01', 0)
('minched01', 0)
('phelpne01', 0)
('quinnpa02', 5)
('selmafr01', 5)
('eggleda01', 0)
('fergubo01', 5)
('fleetfr01', 0)
('hatfijo01', 0)
('highadi01', 1)
('millsch01', 

KeyboardInterrupt: 

In [ ]:
# my_df = pd.read_sql_query(query, conn)
# my_df.head()